In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2

In [2]:
vgg = VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

In [3]:
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']


In [4]:
content_model = Model(inputs=vgg.input, outputs=[vgg.get_layer(name).output for name in content_layers])
style_model = Model(inputs=vgg.input, outputs=[vgg.get_layer(name).output for name in style_layers])


In [5]:
def content_loss(content, target):
    return tf.reduce_mean(tf.square(content - target))


In [6]:
def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

In [7]:
def style_loss(style, target):
    style_gram = gram_matrix(style)
    target_gram = gram_matrix(target)
    return tf.reduce_mean(tf.square(style_gram - target_gram))

In [8]:
def total_variation_loss(image):
    x_deltas, y_deltas = image[:, :-1, :-1, :] - image[:, :-1, 1:, :], image[:, :-1, :-1, :] - image[:, 1:, :-1, :]
    return tf.reduce_mean(tf.square(x_deltas)) + tf.reduce_mean(tf.square(y_deltas))


In [9]:
def compute_loss(content_targets, style_targets, image, content_weight, style_weight, tv_weight):
    content_outputs = content_model(image)
    style_outputs = style_model(image)

    content_loss_val = 0
    style_loss_val = 0

    for target, output in zip(content_targets, content_outputs):
        content_loss_val += content_loss(output[0], target)

    for target, output in zip(style_targets, style_outputs):
        style_loss_val += style_loss(output[0], target)

    content_loss_val *= content_weight / len(content_outputs)
    style_loss_val *= style_weight / len(style_outputs)
    tv_loss_val = total_variation_loss(image) * tv_weight

    total_loss = content_loss_val + style_loss_val + tv_loss_val
    return total_loss

In [10]:
def initialize_image(content_image):
    image = tf.Variable(content_image, dtype=tf.float32)
    return image

In [11]:
def style_transfer(content_image, style_image, num_iterations=1000, content_weight=1e3, style_weight=1e-2, tv_weight=1e-2, learning_rate=5):
    content_image = np.expand_dims(content_image, axis=0)
    style_image = np.expand_dims(style_image, axis=0)

    content_targets = content_model(content_image)
    style_targets = style_model(style_image)

    image = initialize_image(content_image)

    optimizer = Adam(learning_rate=learning_rate, beta_1=0.99, epsilon=1e-1)

    for i in range(num_iterations):
        with tf.GradientTape() as tape:
            loss = compute_loss(content_targets, style_targets, image, content_weight, style_weight, tv_weight)
        grads = tape.gradient(loss, image)
        optimizer.apply_gradients([(grads, image)])

        image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=255.0))

        if i % 100 == 0:
            print(f"Iteration: {i}, Loss: {loss}")

    return image.numpy()

In [12]:
def load_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise Exception(f"Image '{image_path}' not found or cannot be read.")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = preprocess_input(image)
        return image
    except Exception as e:
        print(f"Error loading image '{image_path}': {e}")
        return None

In [13]:
content_image_path = r"C:\Users\baddu\Desktop\python projects\major project\pics\pics2\content_image.jpg"
style_image_path = r"C:\Users\baddu\Desktop\python projects\major project\pics\pics2\style_image.jpg"

In [14]:
content_image = load_image(content_image_path)
style_image = load_image(style_image_path)

In [16]:
if content_image is not None and style_image is not None:
    # Resize images for memory efficiency
    content_image = cv2.resize(content_image, (256, 256))
    style_image = cv2.resize(style_image, (256, 256))

    # Perform Style Transfer
    generated_image = style_transfer(content_image, style_image)

    # Post-process the generated image
    generated_image = np.clip(generated_image[0], 0, 255).astype('uint8')
    generated_image = cv2.cvtColor(generated_image, cv2.COLOR_RGB2BGR)

    # Save the result
    cv2.imwrite(r"C:\Users\baddu\Desktop\python projects\major project\pics\pics2\out.jpg", generated_image)
else:
    print("Failed to load content or style image.")


Iteration: 0, Loss: 1137089920.0
Iteration: 100, Loss: 69517112.0
Iteration: 200, Loss: 35556044.0
Iteration: 300, Loss: 23136060.0
Iteration: 400, Loss: 17193094.0
Iteration: 500, Loss: 14121378.0
Iteration: 600, Loss: 12391549.0
Iteration: 700, Loss: 11327425.0
Iteration: 800, Loss: 10600011.0
Iteration: 900, Loss: 10058615.0


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
